# LOL Models

## Data Export
For LOL modeling is done for performance score because that is the per game score which
translates better across different leagues
```
DB_FILE=${FANTASY_HOME}/lol_hist_2014-2022.scored.db
SEASONS="2016 2017 2018 2019 2020 2021 2022"

# player data
dumpdata.sc --seasons $SEASONS --progress ${DB_FILE} --no_teams \
    --stats "*" --target_calc_stats "*performance_score" \
    --player_team_stats "*" --opp_team_stats "*" \
    --hist_recent_games 3 --hist_recent_mode ma \
    --format parquet -f lol_player.pq

# team data
dumpdata.sc --seasons $SEASONS --progress ${DB_FILE} --no_players \
    --stats "*" --target_calc_stats "*performance_score" \
    --opp_team_stats "*" --hist_extra match_win --current_extra match_win \
    --hist_recent_games 3 --hist_recent_mode ma \
    --format parquet -f lol_team.pq
```

In [ ]:
RANDOM_SEED = 1
TRAINING_TIME = 600
VALIDATION_SEASON = 2022
RECENT_GAMES = 3
REUSE_EXISTING = False
MISSING_DATA_THRESHOLD = .07

In [ ]:
import sys

sys.path.append("..")

from fantasy_py import PlayerOrTeam
from train_test import load_data, model_and_test

TARGET = ("extra", "match_win")
MODEL_NAME = "LOL-team-win"
DATA_FILENAME = "lol_team.pq"
P_OR_T = PlayerOrTeam.TEAM
INCLUDE_POS = None
TRAINING_SEASONS = [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
COLS_DROP_FILTERS = None
TRAINING_TIME = 600

# TARGET = ("calc", "dk_performance_score")
# MODEL_NAME = "LOL-team-DK"
# DATA_FILENAME = "lol_team.pq"
# P_OR_T = PlayerOrTeam.TEAM
# INCLUDE_POS = None
# TRAINING_SEASONS = [2016, 2017, 2018, 2019, 2020, 2021]
# COLS_DROP_FILTERS = ["extra:match_win"]
# TRAINING_TIME = 600

# TARGET = ("calc", "dk_performance_score")
# MODEL_NAME = "LOL-player-DK"
# DATA_FILENAME = "lol_player.pq"
# P_OR_T = PlayerOrTeam.PLAYER
# INCLUDE_POS = True
# TRAINING_SEASONS = [2016, 2017, 2018, 2019, 2020, 2021]
# COLS_DROP_FILTERS = None
# TRAINING_TIME = 900


raw_df, tt_data, one_hot_stats = load_data(
    DATA_FILENAME,
    TARGET,
    VALIDATION_SEASON,
    seed=RANDOM_SEED,
    include_position=INCLUDE_POS,
    col_drop_filters=COLS_DROP_FILTERS,
)

assert one_hot_stats is None

# for automl_type in ["dummy", "tpot", "autosk"]:
for automl_type in ["tpot"]:
    model = model_and_test(
        MODEL_NAME,
        VALIDATION_SEASON,
        tt_data,
        TARGET,
        TRAINING_TIME,
        automl_type,
        P_OR_T,
        RECENT_GAMES,
        TRAINING_SEASONS,
        seed=RANDOM_SEED,
        raw_df=raw_df,
        reuse_existing=REUSE_EXISTING,
    )